# Tensorus: Getting Started

Welcome to the Tensorus Getting Started tutorial! This notebook will walk you through the basics of installing Tensorus, starting the server, and performing some basic operations.

Tensorus is a production-ready, agentic tensor database that provides 10-100x performance improvements over traditional tensor storage solutions.

## 1. Installation

First, let's install Tensorus using pip. If you have followed the setup instructions in the main `README.md`, you may already have this installed.

In [ ]:
# Install Tensorus and required dependencies
!pip install -e .
!pip install requests numpy torch

## 2. Start the Tensorus Server

Now, let's start the Tensorus server. You can do this from your terminal using the `tensorus` CLI or directly with uvicorn.

In [ ]:
# Start the Tensorus server (run this in a separate terminal)
# Option 1: Using the CLI
# !tensorus start

# Option 2: Using uvicorn directly
# !python -m uvicorn tensorus.api:app --reload --host 127.0.0.1 --port 7860

print("Please start the Tensorus server in a separate terminal using one of the commands above.")
print("The server will be available at: http://127.0.0.1:7860")
print("API documentation will be available at: http://127.0.0.1:7860/docs")

## 3. Interacting with the Tensorus API

Now that the server is running, we can interact with it using the Python client with proper error handling.

In [ ]:
import requests
import numpy as np
import json
from typing import Dict, Any, Optional

TENSORUS_API_URL = "http://127.0.0.1:7860"

def check_server_status():
    """Check if the Tensorus server is running."""
    try:
        response = requests.get(f"{TENSORUS_API_URL}/health", timeout=5)
        return response.status_code == 200
    except requests.exceptions.RequestException:
        return False

def create_dataset(name: str) -> Dict[str, Any]:
    """Create a new dataset."""
    try:
        response = requests.post(f"{TENSORUS_API_URL}/datasets/create", json={"name": name})
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error creating dataset: {e}")
        return {"error": str(e)}

def ingest_tensor(dataset_name: str, tensor: np.ndarray, metadata: Dict[str, Any]) -> Dict[str, Any]:
    """Ingest a tensor into a dataset."""
    try:
        shape = list(tensor.shape)
        dtype = str(tensor.dtype)
        data = tensor.tolist()
        payload = {"shape": shape, "dtype": dtype, "data": data, "metadata": metadata}
        response = requests.post(f"{TENSORUS_API_URL}/datasets/{dataset_name}/ingest", json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error ingesting tensor: {e}")
        return {"error": str(e)}

def get_dataset(dataset_name: str) -> Dict[str, Any]:
    """Retrieve dataset information."""
    try:
        response = requests.get(f"{TENSORUS_API_URL}/datasets/{dataset_name}/fetch")
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching dataset: {e}")
        return {"error": str(e)}

def add_tensors(dataset_name: str, tensor1_id: str, tensor2_id: str) -> Dict[str, Any]:
    """Add two tensors together."""
    try:
        payload = {
            "input1": {"dataset_name": dataset_name, "record_id": tensor1_id}, 
            "input2": {"dataset_name": dataset_name, "record_id": tensor2_id}
        }
        response = requests.post(f"{TENSORUS_API_URL}/ops/add", json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error adding tensors: {e}")
        return {"error": str(e)}

# Check if server is running
if check_server_status():
    print("✅ Tensorus server is running!")
else:
    print("❌ Tensorus server is not running. Please start it first.")

### Create a Dataset

Let's create our first dataset to store tensors.

In [ ]:
# Create a new dataset
result = create_dataset("my_first_dataset")
print("Dataset creation result:")
print(json.dumps(result, indent=2))

### Ingest some Tensors

Now let's create and ingest some sample tensors with rich metadata.

In [ ]:
# Create sample tensors
tensor1 = np.array([[1, 2], [3, 4]], dtype=np.float32)
tensor2 = np.array([[5, 6], [7, 8]], dtype=np.float32)

print(f"Tensor 1 shape: {tensor1.shape}, dtype: {tensor1.dtype}")
print(f"Tensor 1 data:\n{tensor1}")
print()
print(f"Tensor 2 shape: {tensor2.shape}, dtype: {tensor2.dtype}")
print(f"Tensor 2 data:\n{tensor2}")

# Ingest tensors with metadata
result1 = ingest_tensor("my_first_dataset", tensor1, {
    "name": "tensor1", 
    "description": "First sample tensor",
    "created_by": "tutorial",
    "version": 1
})

result2 = ingest_tensor("my_first_dataset", tensor2, {
    "name": "tensor2", 
    "description": "Second sample tensor",
    "created_by": "tutorial",
    "version": 1
})

print("\nIngestion results:")
print("Tensor 1:", json.dumps(result1, indent=2))
print("Tensor 2:", json.dumps(result2, indent=2))

### View the Dataset

Let's examine what's stored in our dataset.

In [ ]:
# Fetch dataset information
dataset_info = get_dataset("my_first_dataset")
print("Dataset information:")
print(json.dumps(dataset_info, indent=2))

# Display summary statistics
if "data" in dataset_info and isinstance(dataset_info["data"], list):
    print(f"\n📊 Dataset Summary:")
    print(f"   Total tensors: {len(dataset_info['data'])}")
    for i, tensor_info in enumerate(dataset_info["data"]):
        if "metadata" in tensor_info:
            name = tensor_info["metadata"].get("name", f"tensor_{i}")
            print(f"   - {name}: ID {tensor_info.get('record_id', 'unknown')}")

### Perform an Operation

Now, let's add the two tensors together using Tensorus operations. We need to get their record IDs first.

In [ ]:
# Get dataset and extract tensor IDs
dataset = get_dataset("my_first_dataset")

if "data" in dataset and len(dataset["data"]) >= 2:
    tensor1_id = dataset["data"][0]["record_id"]
    tensor2_id = dataset["data"][1]["record_id"]
    
    print(f"Tensor 1 ID: {tensor1_id}")
    print(f"Tensor 2 ID: {tensor2_id}")
    
    # Perform tensor addition
    addition_result = add_tensors("my_first_dataset", tensor1_id, tensor2_id)
    print("\n🧮 Tensor Addition Result:")
    print(json.dumps(addition_result, indent=2))
    
    # Expected result should be [[6, 8], [10, 12]]
    print("\n✨ Expected result: [[6, 8], [10, 12]]")
else:
    print("❌ Could not find enough tensors in the dataset for the operation.")

## 4. Exploring More Features

Let's explore some additional Tensorus capabilities.

In [ ]:
# Create a larger tensor for demonstration
large_tensor = np.random.rand(100, 100).astype(np.float32)
print(f"Created large tensor with shape: {large_tensor.shape}")

# Ingest with rich metadata
large_result = ingest_tensor("my_first_dataset", large_tensor, {
    "name": "large_random_tensor",
    "description": "100x100 random tensor for performance testing",
    "tensor_type": "random",
    "size_mb": large_tensor.nbytes / (1024 * 1024),
    "created_by": "tutorial",
    "version": 1
})

print("\nLarge tensor ingestion result:")
print(json.dumps(large_result, indent=2))

## Next Steps

Congratulations! 🎉 You have successfully:

- ✅ Installed Tensorus
- ✅ Started the server
- ✅ Created a dataset
- ✅ Ingested tensors with metadata
- ✅ Performed tensor operations
- ✅ Explored performance with larger tensors

### What's Next?

1. **Advanced Features**: Check out `02_advanced_usage.ipynb` for:
   - Natural Language Queries (NQL)
   - Agent Framework
   - Vector Database capabilities

2. **API Documentation**: Visit `http://127.0.0.1:7860/docs` for complete API reference

3. **Performance**: Tensorus provides 10-100x performance improvements over traditional tensor storage

4. **Production Deployment**: See the main README.md for Docker deployment options

### Key Benefits You've Experienced:
- 🚀 **Fast tensor operations** with automatic optimization
- 🧠 **Rich metadata** support for tensor organization
- 🔍 **Easy querying** and retrieval
- 📊 **Scalable architecture** for production workloads